# BB84 Quantum key distribution

# Normal state

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

def step1_alice_prepares_photons(n_bits):
    """
    Step 1: Alice randomly chooses photons in both rectilinear and diagonal bases
    """
    print("=== STEP 1: Alice prepares random bits and bases ===")
    
    # Alice chooses random bits and bases
    alice_bits = [random.randint(0, 1) for _ in range(n_bits)]
    alice_bases = [random.randint(0, 1) for _ in range(n_bits)]  # 0=rectilinear (+), 1=diagonal (x)
    
    print(f"Alice's random bits:  {alice_bits}")
    print(f"Alice's random bases: {alice_bases} (0=rectilinear, 1=diagonal)")
    print()
    
    return alice_bits, alice_bases

# Encoding function (alternative approach)
def encode_message(bits, bases):
    """
    Alternative encoding: prepare states directly in chosen basis
    Z-basis (0): |0⟩ or |1⟩
    X-basis (1): |+⟩ = H|0⟩ or |-⟩ = HX|0⟩
    """
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        # Z-basis: apply X if bit=1
        if basis == 0:
            if bit == 1:
                qc.x(0)
        # X-basis: prepare state by applying H & X
        else:
            if bit == 0:
                qc.h(0)  # |+⟩ state
            else:
                qc.x(0)
                qc.h(0)  # |-⟩ state
        circuits.append(qc)
    return circuits

def step2_alice_sends_photons(alice_bits, alice_bases):
    """
    Step 2: Alice records photon states and sends them to Bob
    """
    print("=== STEP 2: Alice encodes and sends photons ===")
    
    # Use the alternative encoding function
    alice_photons = encode_message(alice_bits, alice_bases)
    
    # Show encoding for each photon
    for i in range(len(alice_bits)):
        basis_name = "Z-basis" if alice_bases[i] == 0 else "X-basis"
        if alice_bases[i] == 0:  # Z-basis
            state_name = "|0⟩" if alice_bits[i] == 0 else "|1⟩"
        else:  # X-basis
            state_name = "|+⟩" if alice_bits[i] == 0 else "|-⟩"
        
        print(f"Photon {i}: bit={alice_bits[i]} → {state_name} in {basis_name}")
    
    print(f"\nAlice sends {len(alice_photons)} photons to Bob")
    print()
    
    return alice_photons

def step3_bob_measures_photons(alice_photons):
    """
    Step 3: Bob receives photons and randomly measures them in rectilinear or diagonal basis
    """
    print("=== STEP 3: Bob measures photons with random bases ===")
    
    n_bits = len(alice_photons)
    bob_bases = [random.randint(0, 1) for _ in range(n_bits)]
    bob_measured_bits = []
    
    print(f"Bob's random bases:   {bob_bases} (0=rectilinear, 1=diagonal)")
    
    simulator = AerSimulator()
    
    for i in range(n_bits):
        # Copy Alice's photon
        qc = alice_photons[i].copy()
        
        # Bob applies his measurement basis
        if bob_bases[i] == 1:  # Diagonal measurement
            qc.h(0)  # Rotate to diagonal basis
        
        # Measure the photon
        qc.measure(0, 0)
        
        # Run simulation
        job = simulator.run(transpile(qc, simulator), shots=1)
        result = job.result()
        counts = result.get_counts()
        measured_bit = int(list(counts.keys())[0])
        
        bob_measured_bits.append(measured_bit)
        
        basis_name = "rectilinear" if bob_bases[i] == 0 else "diagonal"
        print(f"Photon {i}: Bob measures {measured_bit} using {basis_name} basis")
    
    print(f"\nBob's measured bits:  {bob_measured_bits}")
    print()
    
    return bob_bases, bob_measured_bits

def step4_bob_announces_bases(bob_bases):
    """
    Step 4: Bob announces his measurement bases (but not the results)
    """
    print("=== STEP 4: Bob announces his measurement bases ===")
    print(f"Bob publicly announces his bases: {bob_bases}")
    print("(Bob does NOT announce his measurement results)")
    print()
    
    return bob_bases

def step5_alice_announces_matching(alice_bases, bob_bases):
    """
    Step 5: Alice announces which measurements used the correct (matching) bases
    """
    print("=== STEP 5: Alice announces matching bases ===")
    
    matching_indices = []
    for i in range(len(alice_bases)):
        if alice_bases[i] == bob_bases[i]:
            matching_indices.append(i)
    
    print(f"Alice's bases:        {alice_bases}")
    print(f"Bob's bases:          {bob_bases}")
    print(f"Matching positions:   {matching_indices}")
    print()
    
    return matching_indices

def step6_create_shared_key(alice_bits, bob_measured_bits, matching_indices):
    """
    Step 6: Alice and Bob discard mismatched measurements and create bit string
    """
    print("=== STEP 6: Create shared key from matching measurements ===")
    
    alice_key = [alice_bits[i] for i in matching_indices]
    bob_key = [bob_measured_bits[i] for i in matching_indices]
    
    print(f"Alice's key: {alice_key}")
    print(f"Bob's key:   {bob_key}")
    
    # Check for errors
    errors = sum(1 for i in range(len(alice_key)) if alice_key[i] != bob_key[i])
    error_rate = errors / len(alice_key) if alice_key else 0
    
    print(f"Errors: {errors}/{len(alice_key)} = {error_rate:.2%}")
    print()
    
    return alice_key, bob_key, error_rate

def run_bb84_protocol(n_bits=8):
    """
    Run the complete BB84 protocol
    """
    print("🔐 BB84 Quantum Key Distribution Protocol")
    print("=" * 50)
    print()
    
    # Step 1: Alice prepares
    alice_bits, alice_bases = step1_alice_prepares_photons(n_bits)
    
    # Step 2: Alice sends photons
    alice_photons = step2_alice_sends_photons(alice_bits, alice_bases)
    
    # Step 3: Bob measures
    bob_bases, bob_measured_bits = step3_bob_measures_photons(alice_photons)
    
    # Step 4: Bob announces bases
    step4_bob_announces_bases(bob_bases)
    
    # Step 5: Alice announces matching
    matching_indices = step5_alice_announces_matching(alice_bases, bob_bases)
    
    # Step 6: Create shared key
    alice_key, bob_key, error_rate = step6_create_shared_key(alice_bits, bob_measured_bits, matching_indices)
    
    # Summary
    print("=== PROTOCOL COMPLETE ===")
    if error_rate == 0:
        print("✅ Success! Alice and Bob have identical keys.")
    else:
        print(f"⚠️  Error detected: {error_rate:.2%} - possible eavesdropping!")
    
    return alice_key, bob_key, error_rate

# Example usage
if __name__ == "__main__":
    # Run BB84 protocol with 8 bits
    alice_key, bob_key, error_rate = run_bb84_protocol(n_bits=8)
    
    print(f"\nFinal shared key length: {len(alice_key)} bits")
    print(f"Key security: {'SECURE' if error_rate < 0.1 else 'COMPROMISED'}")

### 